# Bank Project - Preprocessing

In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from collections import Counter

## Fetch Data

In [4]:
df = pd.read_csv('./data/bank-full.csv', sep=';')

## Transform 'y' into 'target'

 + Useful for quantifying the target

In [5]:
df['target']= [1 if y=='yes' else 0 for y in df.y]

Preprocessing FUnction

In [285]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
#Custom transformer we wrote to engineer features ( bathrooms per bedroom and/or how old the house is in 2019  ) 
#passed as boolen arguements to its constructor
class TargetTransformer(BaseEstimator, TransformerMixin):
#Class Constructor
    def __init__(self):
        self = self

#Return self, nothing else to do here
    def fit( self, X ):
        return self 

#Custom transform method we wrote that creates aformentioned features and drops redundant ones 
    def transform(self, X):
        #Check if needed 
        X = np.array([1 if y=='yes' else 0 for y in X])
        #returns a numpy array
        return X.reshape(-1,1)

class LogPlus1Transformer(BaseEstimator, TransformerMixin):
#Class Constructor
    def __init__(self):
        self=self
    
    def fit(self, X):
        return(self)
    
    def transform(self, X):
        X = np.log(X+1)
        return(X.values)

class CampaignTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
    
    def fit(self, X):
        return(self)

    def transform(self, X):
        X = np.array(pd.cut(X, np.quantile(X, [0,0.25,0.5,0.75,1]), duplicates='drop', include_lowest=True).astype(str))
        return(X.reshape(-1,1))

In [286]:
cti = CampaignTransformer()

h = (cti.fit_transform(df.campaign))

h

array([['(0.999, 2.0]'],
       ['(0.999, 2.0]'],
       ['(0.999, 2.0]'],
       ...,
       ['(3.0, 63.0]'],
       ['(3.0, 63.0]'],
       ['(0.999, 2.0]']], dtype=object)

In [257]:
cat_features = ['job','marital','education','default','housing','load','contact','poutcome','month']

In [258]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
    [("target",TargetTransformer(),'y'),
    ("logplus1",LogPlus1Transformer(),['duration','age']),
    ("campaign",CampaignTransformer(),'campaign'),
    ('ohe',OneHotEncoder(sparse=False),cat_features)])

df_trans = ct.fit_transform(df)
df_trans

ValueError: 'load' is not in list

In [252]:
pd.DataFrame(df_trans).head()

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
0,0,5.56834,4.07754,"(0.999, 2.0]",0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
1,0,5.02388,3.80666,"(0.999, 2.0]",0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
2,0,4.34381,3.52636,"(0.999, 2.0]",0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
3,0,4.5326,3.8712,"(0.999, 2.0]",0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
4,0,5.2933,3.52636,"(0.999, 2.0]",0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
